<a href="https://colab.research.google.com/github/harveenchadha/bol/blob/main/demos/hf/hindihf_with_lm_vakyansh_hindi_him_4200_evaluation_common_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture

!pip -q install transformers datasets torchaudio jiwer pyctcdecode https://github.com/kpu/kenlm/archive/master.zip

In [4]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM
import re


wer = load_metric("wer")

processor = Wav2Vec2ProcessorWithLM.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model.to("cuda")

Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd)


Downloading:   0%|          | 0.00/863 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [35]:

from datasets import  Audio
resampler = torchaudio.transforms.Resample(48_000, 16_000)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    batch["speech"] = batch["audio"]["array"]
    return batch

test_dataset = load_dataset("common_voice", "hi", split="test")
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
processor = Wav2Vec2ProcessorWithLM.from_pretrained('/content/vakyansh-wav2vec2-hindi-him-4200')
def evaluate(batch):
    
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    # with torch.no_grad():
    #     logits = model(inputs.input_values.to("cuda")).logits

    #     pred_ids = torch.argmax(logits, dim=-1)
    #     batch["pred_strings"] = processor.batch_decode(pred_ids, skip_special_tokens=True)
    #     return batch
    
    with torch.no_grad():
        logits = model(**inputs.to('cuda')).logits
    int_result = processor.batch_decode(logits.cpu().numpy())
    output = []
    for item in int_result.text:
        output.append(item.replace('<s>',''))
    batch["pred_strings"] =  output
    return batch


result = test_dataset.map(evaluate, batched=True, batch_size=8)


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd)
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd/cache-3d7a1251c13576ec.arrow


  0%|          | 0/16 [00:00<?, ?ba/s]

In [36]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

WER: 53.132251


In [30]:
!git clone https://huggingface.co/Harveenchadha/vakyansh-wav2vec2-hindi-him-4200

Cloning into 'vakyansh-wav2vec2-hindi-him-4200'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 65 (delta 26), reused 0 (delta 0)
Unpacking objects: 100% (65/65), done.


In [32]:
!rm vakyansh-wav2vec2-hindi-him-4200/language_model/kenLM.arpa
!rm vakyansh-wav2vec2-hindi-him-4200/language_model/unigrams.txt

rm: cannot remove 'vakyansh-wav2vec2-hindi-him-4200/language_model/kenLM.arpa': No such file or directory


In [33]:
!wget https://storage.googleapis.com/vakyansh-open-models/models/hindi/hi-IN/lm.binary -P vakyansh-wav2vec2-hindi-him-4200/language_model/

--2022-01-22 17:03:35--  https://storage.googleapis.com/vakyansh-open-models/models/hindi/hi-IN/lm.binary
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.70.128, 64.233.181.128, 64.233.182.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.70.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1499997273 (1.4G) [application/octet-stream]
Saving to: ‘vakyansh-wav2vec2-hindi-him-4200/language_model/lm.binary’

lm.binary           100%[===================>]   1.40G   170MB/s    in 9.3s    

2022-01-22 17:03:44 (154 MB/s) - ‘vakyansh-wav2vec2-hindi-him-4200/language_model/lm.binary’ saved [1499997273/1499997273]



In [34]:
!wget https://storage.googleapis.com/vakyaansh-open-models/hindi/unigrams.txt -P vakyansh-wav2vec2-hindi-him-4200/language_model/

--2022-01-22 17:12:15--  https://storage.googleapis.com/vakyaansh-open-models/hindi/unigrams.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.103.128, 142.250.128.128, 142.251.6.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.103.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82330121 (79M) [text/plain]
Saving to: ‘vakyansh-wav2vec2-hindi-him-4200/language_model/unigrams.txt’

unigrams.txt        100%[===================>]  78.52M  42.2MB/s    in 1.9s    

2022-01-22 17:12:17 (42.2 MB/s) - ‘vakyansh-wav2vec2-hindi-him-4200/language_model/unigrams.txt’ saved [82330121/82330121]

